In [1]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, default_data_collator, DataCollatorForLanguageModeling
from datasets import load_dataset
from arguments import DataTrainingArguments, ModelArguments, TrainingArguments

# Initialize the tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-base', legacy=True)

# Initialize the data arguments
data_args = DataTrainingArguments(
    datasets='floorplan',
    data_dir=None,
    train_split='train',
    max_seq_length=512,
    max_output_seq_length=512,
    max_seq_length_eval=512,
    max_output_seq_length_eval=512,
    exp='full',
    boundary_in_where='Encoder'
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
model_args = ModelArguments(model_name_or_path='t5-base', config_name=None, tokenizer_name=None, cache_dir=None)

training_args = TrainingArguments()

In [3]:
training_args.do_train

False

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
    padding="max_length",
)

In [5]:
dataset = load_dataset('floorplan', data_args, split='train',
                         max_input_length=data_args.max_seq_length, max_output_length=data_args.max_output_seq_length,
                         tokenizer=tokenizer, seed=1, train_subset=data_args.train_subset)

Example input sententece: Balcony 1 is about 8 foot X 2 foot . This is located right off of the kitchen . Balcony 2 is about 10 foot X 3 foot . This is facing north of the house . The bathroom is about 8 foot X 6 foot . It is located between the living room and the kitchen . Common room 1 is about 10 foot X 12 foot . The common rooms are located diagonal from each other . Common room 2 is about 10 foot X 10 foot . The common rooms are located diagonal from each other . The kitchen is a 10 foot X 10 foot room . It looks like it is opened up to the living room . The living room is located on the east side of the house . It is shaped like a T and it's the biggest room . It is approximately 14 feet X 32 feet . Each room that you come out of you will enter the living room . The master is about 10 feet X 15 feet . It is located far west of the house . It is a rectangle shape .+ 43 51 213 205 - 196 202 218 180 - 175 79 202 52 - 170 122 224 68 - 132 83 165 50 - 42 115 106 50 - 42 203 57 188
Ex

AttributeError: 'list' object has no attribute 'size'

In [18]:
model = T5ForConditionalGeneration.from_pretrained("t5-base")

In [19]:
# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False, mlm_probability=0.15)

In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=floorplan_dataset,
    data_collator=default_data_collator,
)

In [24]:
trainer.train()

ValueError: expected sequence of length 28 at dim 1 (got 24)

In [6]:
import json

file_path = 'data/floorplan/floorplan_train.json'

with open(file_path, 'r') as f:
    data = json.load(f)

In [10]:
data[0]['rooms']

[{'room_type': 'master room',
  'x': 102,
  'y': 77,
  'h': 59,
  'w': 42,
  'x_min': 73,
  'y_min': 56,
  'x_max': 131,
  'y_max': 97,
  'near_x_min': -1,
  'near_y_min': -1,
  'near_x_max': -1,
  'near_y_max': -1,
  'relation': [],
  'location': 'south west corner',
  'size': '150',
  'aspect ratio': '5 over 7',
  'private': False},
 {'room_type': 'common room 1',
  'x': 154,
  'y': 96,
  'h': 34,
  'w': 47,
  'x_min': 137,
  'y_min': 73,
  'x_max': 170,
  'y_max': 119,
  'near_x_min': -1,
  'near_y_min': -1,
  'near_x_max': -1,
  'near_y_max': -1,
  'relation': [],
  'location': 'north west corner',
  'size': '100',
  'aspect ratio': '11 over 8',
  'private': False},
 {'room_type': 'balcony 1',
  'x': 192,
  'y': 139,
  'h': 33,
  'w': 12,
  'x_min': 176,
  'y_min': 133,
  'x_max': 208,
  'y_max': 144,
  'near_x_min': -1,
  'near_y_min': -1,
  'near_x_max': -1,
  'near_y_max': -1,
  'relation': [],
  'location': 'north side',
  'size': '25',
  'aspect ratio': '4 over 11',
  'private

In [11]:
def format_short_output(example) -> str:
    string = ''
    start_token = '['
    end_token = ']'
    sep = '|'
    for room in example['rooms']:
        # string += f'{start_token} {room.type} {sep} x coordinate = {str(room.x)} {sep} y coordinate = {str(room.y)} {sep} height = {str(room.h)} {sep} width = {str(room.w)} {end_token} '
        string += f'{room.type} {str(room.x_min)} {str(room.y_min)} {str(room.x_max)} {str(room.y_max)} '
    return string

format_short_output(data[0])

AttributeError: 'dict' object has no attribute 'type'

In [13]:
from datasets import FloorplanDataset

floorplan_dataset = FloorplanDataset(tokenizer=tokenizer,max_input_length=512, max_output_length=512, data_args=data_args, seed=1)

In [14]:
len(floorplan_dataset.features)

2246

In [15]:
type(floorplan_dataset)

datasets.FloorplanDataset

In [16]:
data_args.output_format_type

'short'

In [25]:
floorplan_dataset.features[0]

InputFeatures(input_ids=[4011, 1018, 63, 209, 19, 81, 505, 2418, 3, 4, 204, 2418, 3, 5, 100, 19, 1069, 269, 326, 13, 8, 1228, 3, 5, 4011, 1018, 63, 204, 19, 81, 335, 2418, 3, 4, 220, 2418, 3, 5, 100, 19, 5008, 3457, 13, 8, 629, 3, 5, 37, 2582, 19, 81, 505, 2418, 3, 4, 431, 2418, 3, 5, 94, 19, 1069, 344, 8, 840, 562, 11, 8, 1228, 3, 5, 7155, 562, 209, 19, 81, 335, 2418, 3, 4, 586, 2418, 3, 5, 37, 1017, 2801, 33, 1069, 26184, 45, 284, 119, 3, 5, 7155, 562, 204, 19, 81, 335, 2418, 3, 4, 335, 2418, 3, 5, 37, 1017, 2801, 33, 1069, 26184, 45, 284, 119, 3, 5, 37, 1228, 19, 3, 9, 335, 2418, 3, 4, 335, 2418, 562, 3, 5, 94, 1416, 114, 34, 19, 2946, 95, 12, 8, 840, 562, 3, 5, 37, 840, 562, 19, 1069, 30, 8, 5727, 596, 13, 8, 629, 3, 5, 94, 19, 3, 6489, 114, 3, 9, 332, 11, 34, 31, 7, 8, 2630, 562, 3, 5, 94, 19, 3241, 968, 1922, 3, 4, 3538, 1922, 3, 5, 1698, 562, 24, 25, 369, 91, 13, 25, 56, 2058, 8, 840, 562, 3, 5, 37, 2325, 19, 81, 335, 1922, 3, 4, 627, 1922, 3, 5, 94, 19, 1069, 623, 4653, 13, 8, 

In [41]:
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, features):
        self.features = features

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = self.features[idx]
        if feature is None:
            return torch.zeros((1, )), torch.zeros((1, ))
        return {
            'input_ids': feature.input_ids,
            'attention_mask': feature.attention_mask,
            'boundary_ids': feature.boundary_ids,
            'boundary_mask': feature.boundary_mask,
            'num_rooms': feature.num_rooms,
            'regr_labels': feature.regr_labels,
            'decoder_attention_mask': feature.decoder_attention_mask,
        }, feature.label_ids

dataset = MyDataset(floorplan_dataset.features)

In [44]:
dataloader = DataLoader(dataset, batch_size=32)

for batch in dataloader:
    print(batch)
    break

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>

In [43]:
from torch.optim import AdamW

# Initialize the optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
for epoch in range(2):
    for batch in dataloader:
        # Unpack the batch
        inputs, targets = batch

        # Move the inputs and targets to the same device as the model
        inputs = {name: tensor.to(model.device) for name, tensor in inputs.items()}
        targets = targets.to(model.device)

        # Forward pass: feed the inputs to the model and get the outputs
        outputs = model(**inputs, labels=targets)

        # Compute the loss
        loss = outputs.loss

        # Backward pass: compute the gradients of the loss w.r.t. the model's parameters
        loss.backward()

        # Update the model's parameters
        optimizer.step()

        # Clear the gradients
        optimizer.zero_grad()

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'NoneType'>